# 添加算子映射关系初级教程
`Linux` `Ascend` `GPU` `CPU` `模型迁移` `初级`

[![](https://gitee.com/mindspore/docs/raw/master/tutorials/training/source_zh_cn/_static/logo_source.png)](https://gitee.com/mindspore/mindinsight/blob/master/mindinsight/mindconverter/tutorial/add_operator_mapper_base_tutorial.ipynb)

## 概述

MindConverter工具基于ONNX模型进行脚本转换，生成MindSpore脚本和权重文件。因此需要ONNX算子到MindSpore算子的映射关系文件来保证算子之间转换结果的正确性。本文将以简单的算子映射关系为例，来描述添加算子映射关系文件的方法。

## 环境准备

本案例需安装以下Python三方库：
```bash
pip install mindspore==1.2.0
pip install mindinsight==1.2.0
```

## 算子映射脚本（base.py)结构

In [1]:
import abc


class Mapper(metaclass=abc.ABCMeta):
    """Mapper between third-party-operation and MindSpore."""

    @staticmethod
    @abc.abstractmethod
    def _operation_name_in_ms(**kwargs):
        """Corresponding operation name in MindSpore."""

    @staticmethod
    @abc.abstractmethod
    def _convert_params(**kwargs):
        """Convert third-party-operation's attributes or weights into MindSpore operation's attributes."""

    @staticmethod
    @abc.abstractmethod
    def _convert_trained_weights(**kwargs):
        """Convert third-party-operation's trainable weights into MindSpore operation's."""

    @staticmethod
    @abc.abstractmethod
    def _generate_snippet_template(**kwargs):
        """Generate code template according to node info."""

## 自定义添加算子映射脚本

以`onnx::Gemm`算子为例进行演示。

分别查阅[ONNX算子API文档](https://github.com/onnx/onnx/blob/master/docs/Operators.md)和[MindSpore算子API文档](http://www.mindspore.cn/doc/api_python/zh-CN/master/index.html)，
找到与ONNX算子`onnx::Gemm`功能相同或相近的MindSpore算子`mindspore.nn.Dense`。

|算子名|`onnx::Gemm`|`mindspore.nn.Dense`|
|:----:|:----|:----|
|算法实现|`Y = alpha*A'*B'+beta*C`|`output = activation(inputs*kernel+bias)`|
|参数|`alpha`: optional<br>`beta`: optional<br>`transA`: optional<br>`transB`: optional|`in_channels`: required<br>`out_channels`: required<br>`weight_init`: optional<br>`bias_init`: optional<br>`has_bias`: optional<br>`activation`: optional|
|输入|`A`: required<br>`B`: required<br>`C`: optional|`input`: required|
|输出|`Y`|`output`|

<br>
依据双方算子中参数（Attributes/Parameters）和输入（Inputs）进行ONNX到MindSpore的算子映射。

In [2]:
import numpy as np

# 导入Mapper基类
from mindinsight.mindconverter.graph_based_converter.mapper.base import ONNXToMindSporeMapper


class DenseMapper(ONNXToMindSporeMapper):
    """Dense mapper."""

    @staticmethod
    def _operation_name_in_ms(*args, **kwargs):
        return "nn.Dense"  # MindSpore中对应的算子名

    @staticmethod
    def _convert_params(**kwargs):
        """
        参数迁移相关方法，该方法返回的参数将在生成的MindSpore脚本中以
        `OP(dict_key_0=dict_value_0, dict_key_1=dict_value_1, ...)`的形式
        定义算子，因此需要保证dict_key_x与MindSpore算子中的参数名相同。

        Args:
            kwargs: Data for converting.
            Struct is `{
                        'weights': [NodeWeight(), NodeWeight(), ...],
                        'params': {
                                    'input_shape': input_shape,
                                    'output_shape': output_shape,
                                    'onnx_attribute_name_0': onnx_attribute_val_0,
                                    ...
                                  }
                        }`
        """

        weights = kwargs['weights']  # 获取ONNX算子的Inputs中的静态Tensor列表
        # 获取Tensor列表中指定序列号的Tensor值，其中序列号与ONNX算子中的Inputs顺序一致。
        weight = DenseMapper._find_val_by_index(0, weights)
        bias = DenseMapper._find_val_by_index(1, weights)
        has_bias = isinstance(bias, np.ndarray)
        in_channels, out_channels = weight.shape
        return {
            'in_channels': in_channels,
            'out_channels': out_channels,
            'has_bias': has_bias
        }

    @staticmethod
    def _convert_trained_weights(**kwargs):
        """
        权重迁移相关方法，该方法返回的权重将会保存在生成的CheckPoint（.ckpt）文件当中
        使生成的MindSpore脚本可以直接加载该权重文件用于重训练或推理。
        详细的内容可参考进阶篇。
        """

        weights = kwargs['weights']
        weight = DenseMapper._find_val_by_index(0, weights)
        bias = DenseMapper._find_val_by_index(1, weights)
        return {
            'weight': {'data': weight},
            'bias': {'data': bias}
        }

将该Mapper脚本命名为`dense_mapper.py`，该命名方式需要和类名（`DenseMapper`)相对应。<br>
并放入 `mindinsight/mindconverter/graph_based_converter/mapper/impl/nn`目录下，该放置目录需要根据对应的MindSpore算子所在的层（`nn`/`ops`）来设置。<br>
最后修改 `mindinsight/mindconverter/graph_based_converter/mapper/onnx_to_ms.json`，
添加 `"onnx::Gemm": "mindinsight.mindconverter.graph_based_converter.mapper.impl.nn.dense_mapper.DenseMapper"`来确定ONNX算子所对应的Mapper脚本文件。

## 验证自定义算子映射脚本

In [3]:
import numpy as np
from mindinsight.mindconverter.graph_based_converter.mapper.base import ONNXToMindSporeMapper
from mindinsight.mindconverter.graph_based_converter.common.code_fragment import NewFragment
from mindinsight.mindconverter.graph_based_converter.third_party_graph.onnx_utils import NodeWeight

def test_mapper(onnx_info):
    """
    Test mapper.

    Args:
        onnx_info (dict): Onnx operator_info. Struct is
                                   {
                                    'op_name': op_name,
                                    'attributes': dict(),
                                    'weights': [NodeWeight(), ...]
                                   }
    """

    template, exchange_msg, outputs_lists, outputs_mapping = \
        ONNXToMindSporeMapper.convert(onnx_info['op_name'],
                                      onnx_info['attributes'],
                                      onnx_info['weights'])

    exchange_msg['var_0']['variable_name'] = 'self_defined_operator'
    exchange_msg['var_0']['inputs'] = ['x']

    fragment = NewFragment(data_entity=exchange_msg, code_template=template, outputs=outputs_lists,
                           outputs_mapping=outputs_mapping)

    code = fragment()
    init_code = code[0]
    construct_code = code[1]
    print('-'*30, 'init_code', '-'*30)
    print('\n'.join(init_code))
    print('-'*30, 'construct_code', '-'*30)
    print('\n'.join(construct_code))

In [4]:
onnx_operator_info = {'op_name': 'onnx::Gemm',
                      'attributes': {'alpha': 1.0,
                                     'beta': 1.0,
                                     'transA': 0,
                                     'transB': 0},
                      'weights': [NodeWeight(weight_name='weight',
                                             weight_location=1,
                                             weight_value=np.ones((10, 3),
                                                                  dtype=np.int)),
                                  NodeWeight(weight_name='bias',
                                             weight_location=2,
                                             weight_value=np.ones((10, 3),
                                                                  dtype=np.int))]}
test_mapper(onnx_operator_info)

------------------------------ init_code ------------------------------
self.self_defined_operator = nn.Dense(in_channels=3, out_channels=10, has_bias=True)
------------------------------ construct_code ------------------------------
opt_self_defined_operator = self.self_defined_operator(x)
